In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests
import urllib.request
import time

In [ ]:
authors = []
dates = []
statements = []
content = []
sources = []
targets = []

In [ ]:
def scrape_website(page_number):
  pgno = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno +'&category=facebook-fact-checks'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  #Get location
  statement_footer = soup.find_all('footer',attrs ={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'})
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' '+ day + ' '+year
    dates.append(date)
    authors.append(full_name)

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)
    atag=i.find_all('a')[0]
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    URL2 = 'https://www.politifact.com' +  atag['href']
    webpage = requests.get(URL2)
    soup2 = BeautifulSoup(webpage.text)
    statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
    content.append(''.join(statement_content[0].text.strip().splitlines()))


  for i in statement_quote:
   atag=i.find_all('a')[0]   

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
      link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
      targets.append(link4)





In [ ]:
def scrape_website1(pgno):
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno +'&category=facebook-fact-checks'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  for i in statement_quote:
   atag=i.find_all('a')[0]
   link2 = i.find_all('a')
   statement_text = link2[0].text.strip()
   URL2 = 'https://www.politifact.com' +  atag['href']
   webpage = requests.get(URL2)
   soup2 = BeautifulSoup(webpage.text)
   statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
   content.append(''.join(statement_content[0].text.strip().splitlines()))

In [ ]:
n = 100
for i in range(1,n):
  scrape_website(i)

In [ ]:
data = pd.DataFrame(columns = ['author','statements','article','source','date','target'])
data['author'] = authors
data['statements']= statements
data['source']= sources
data['article'] = content
data['date']= dates
data['target']= targets

data


,author,statements,article,source,date,target
0,Jeff Cercone,“Joe Biden put his pants on backwards”,Attempts on social media to make President Joe...,Instagram posts,"February 10, 2023",pants-fire
1,Ciara O'Rourke,Gen. Mark Milley “never served in combat.”,"As chairman of the joint chiefs of staff, U.S....",Viral image,"February 10, 2023",false
2,Ciara O'Rourke,Photos show someone in a President Joe Biden m...,"U.S. Sen. Bernie Sanders, I-Vt., drew attentio...",Facebook posts,"February 10, 2023",pants-fire
3,Ciara O'Rourke,A photo shows a deputy U.S. marshal “keeping a...,From U.S. Sen. Mitt Romney of Utah scolding fe...,Facebook posts,"February 10, 2023",false
4,Ciara O'Rourke,A video shows a dog trying to reach her puppie...,The recent earthquake in Turkey and Syria has ...,Viral image,"February 10, 2023",false
...,...,...,...,...,...,...
2995,Samantha Putterman,Says President Donald Trump is selling coronav...,Headlines going around social media are leadin...,Facebook posts,"May 1, 2020",false
2996,Ciara O'Rourke,"Walmart, Amazon, Kroger, Target and Costco “ha...",Questioning why some small businesses around t...,Viral image,"May 1, 2020",false
2997,Samantha Putterman,The existence of a canine coronavirus vaccine ...,"There is currently no vaccine for SARS-CoV-2, ...",Facebook posts,"May 1, 2020",false
2998,Tom Kertscher,The CDC recommends that only people with COVID...,President Donald Trump has said he can’t see h...,Facebook posts,"May 1, 2020",false


In [ ]:
def getBinaryTarget(text):
  if text == 'true':
    return 'REAL'
  else:
    return 'FALSE'  

In [ ]:
def getBinaryNumTarget(text):
  if text == 'true':
    return 1
  else:
    return 0  

In [ ]:
data['BinaryTarget'] = data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [ ]:
data

,author,statements,article,source,date,target,BinaryTarget,BinaryNumTarget
0,Jeff Cercone,“Joe Biden put his pants on backwards”,Attempts on social media to make President Joe...,Instagram posts,"February 10, 2023",pants-fire,FALSE,0
1,Ciara O'Rourke,Gen. Mark Milley “never served in combat.”,"As chairman of the joint chiefs of staff, U.S....",Viral image,"February 10, 2023",false,FALSE,0
2,Ciara O'Rourke,Photos show someone in a President Joe Biden m...,"U.S. Sen. Bernie Sanders, I-Vt., drew attentio...",Facebook posts,"February 10, 2023",pants-fire,FALSE,0
3,Ciara O'Rourke,A photo shows a deputy U.S. marshal “keeping a...,From U.S. Sen. Mitt Romney of Utah scolding fe...,Facebook posts,"February 10, 2023",false,FALSE,0
4,Ciara O'Rourke,A video shows a dog trying to reach her puppie...,The recent earthquake in Turkey and Syria has ...,Viral image,"February 10, 2023",false,FALSE,0
...,...,...,...,...,...,...,...,...
2995,Samantha Putterman,Says President Donald Trump is selling coronav...,Headlines going around social media are leadin...,Facebook posts,"May 1, 2020",false,FALSE,0
2996,Ciara O'Rourke,"Walmart, Amazon, Kroger, Target and Costco “ha...",Questioning why some small businesses around t...,Viral image,"May 1, 2020",false,FALSE,0
2997,Samantha Putterman,The existence of a canine coronavirus vaccine ...,"There is currently no vaccine for SARS-CoV-2, ...",Facebook posts,"May 1, 2020",false,FALSE,0
2998,Tom Kertscher,The CDC recommends that only people with COVID...,President Donald Trump has said he can’t see h...,Facebook posts,"May 1, 2020",false,FALSE,0


In [ ]:
data.to_csv('New_Data.csv')